# Домашнее задание № 2. Мешок слов

## Задание 1 (3 балла)

У векторайзеров в sklearn есть встроенная токенизация на регулярных выражениях. Найдите способо заменить её на кастомную токенизацию

Обучите векторайзер с дефолтной токенизацией и с токенизацией razdel.tokenize. Обучите классификатор с каждым из векторизаторов. Сравните метрики и выберете победителя. 

(в вашей тетрадке должен быть код обучения и все метрики; если вы сдаете в .py файлах то сохраните полученные метрики в отдельном файле или в комментариях)

In [20]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from razdel import tokenize as razdel_tokenize
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import MultinomialNB
import numpy as np


In [21]:
import os 
os.getcwd()


'C:\\Users\\123ha\\comp ling'

In [22]:
def raztok(comment):
    list(razdel_tokenize(comment))
    return [token.text for token in list(razdel_tokenize(comment))]


In [24]:
data = pd.read_csv('labeled.csv')
data['raztokd'] = data.apply(lambda row: raztok(row['comment']), axis =1)
data.head()

,comment,toxic,raztokd
0,"Верблюдов-то за что? Дебилы, бл...\n",1.0,"[Верблюдов-то, за, что, ?, Дебилы, ,, бл, ...]"
1,"Хохлы, это отдушина затюканого россиянина, мол...",1.0,"[Хохлы, ,, это, отдушина, затюканого, россияни..."
2,Собаке - собачья смерть\n,1.0,"[Собаке, -, собачья, смерть]"
3,"Страницу обнови, дебил. Это тоже не оскорблени...",1.0,"[Страницу, обнови, ,, дебил, ., Это, тоже, не,..."
4,"тебя не убедил 6-страничный пдф в том, что Скр...",1.0,"[тебя, не, убедил, 6-страничный, пдф, в, том, ..."


In [26]:
# ваш код
train, test = train_test_split(data, test_size = 0.1, shuffle = True)
train.reset_index(inplace=True)
test.reset_index(inplace=True)


In [27]:
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(train.comment)
X_test = vectorizer.transform(test.comment)
y = train.toxic.values
y_test = test.toxic.values

In [28]:
clf = LogisticRegression(C=0.1, class_weight='balanced')
clf.fit(X, y)
preds = clf.predict(X_test)
print(classification_report(y_test, preds, zero_division=0))

              precision    recall  f1-score   support

         0.0       0.92      0.82      0.87       968
         1.0       0.70      0.85      0.77       474

    accuracy                           0.83      1442
   macro avg       0.81      0.84      0.82      1442
weighted avg       0.85      0.83      0.84      1442



In [29]:
vectorizer = CountVectorizer(analyzer=lambda x: x)
Xr = vectorizer.fit_transform(train.raztokd) #trains the data based on the words tokenized by razdel 
X_testr = vectorizer.transform(test.raztokd)
yr = train.toxic.values
y_test = test.toxic.values

clf = LogisticRegression(C=0.1, class_weight='balanced', max_iter =1000)
clf.fit(Xr, yr)
predsr = clf.predict(X_testr)
print(classification_report(y_test, predsr, zero_division=0))

              precision    recall  f1-score   support

         0.0       0.91      0.84      0.87       968
         1.0       0.71      0.84      0.77       474

    accuracy                           0.84      1442
   macro avg       0.81      0.84      0.82      1442
weighted avg       0.85      0.84      0.84      1442



Токенизация у векторайзеров в sklearn работает лучше, чем токенизация у razdel.tokenize. Хотя разница очень маленькая, мы видим что sklearn с больше precision и выше recall опредляет, какие твиты должны быть классифицированы как "1" (токсичный)
Хотя, каждый раз разница очень маленькый, и значения precision и  recall можно меняться, f1-score всегда выше когда sklearn сам токенизорует комменты

## Задание 2 (3 балла)

Преобразуйте таблицу с абсолютными частотностями в семинарской тетрадке в таблицу с tfidf значениями. (Таблица - https://i.ibb.co/r5Nc2HC/abs-bow.jpg) Формула tfidf есть в семинаре на картнике с пояснениями на английском. 
Считать нужно в питоне. Формат итоговой таблицы может быть любым, главное, чтобы был код и можно было воспроизвести вычисления. 

In [31]:
# ваш код
import re 
import numpy
strings = ['я и ты', 'ты и я', 'я я я и только я','только не я','он']
splitstrings =[]
for string in strings:
    wordss = re.split(' ', string)
    splitstrings.append(wordss)
WOdups =[] #array without duplicates to calculate freqdict
for i in splitstrings:
    WOdups.append(list(dict.fromkeys(i)))
freqdict ={}
for i in WOdups:
    for j in i:
        if j in freqdict:
            freqdict[j]+=1
        if j not in freqdict:
            freqdict[j]=1
numOfDocs = len(strings)
answerArray =[]
for string in splitstrings:
    toadd =[]
    for word in string:
        numOfTimes = 0 
        for word2 in string: 
            if word == word2:
                numOfTimes +=1
        N = numpy.log(numOfDocs/freqdict[word])*numOfTimes
        toadd.append(N)
    answerArray.append(toadd)
for array in answerArray:
    print(array)


[0.22314355131420976, 0.5108256237659907, 0.9162907318741551]
[0.9162907318741551, 0.5108256237659907, 0.22314355131420976]
[0.8925742052568391, 0.8925742052568391, 0.8925742052568391, 0.5108256237659907, 0.9162907318741551, 0.8925742052568391]
[0.9162907318741551, 1.6094379124341003, 0.22314355131420976]
[1.6094379124341003]


## Задание 3 (2 балла)

Обучите 2 любых разных классификатора из семинара. Предскажите токсичность для текстов из тестовой выборки (используйте одну и ту же выборку для обоих классификаторов) и найдите 10 самых токсичных для каждого из классификаторов. Сравните получаемые тексты - какие тексты совпадают, какие отличаются, правда ли тексты токсичные?

Требования к классификаторам:   
а) один должен использовать CountVectorizer, другой TfidfVectorizer  
б) у векторазера должны быть вручную заданы как минимум 5 параметров  
в) у классификатора должно быть задано вручную как минимум 2 параметра  
г)  f1 мера каждого из классификаторов должна быть минимум 0.75  

In [32]:
# ваш код
dataFrame = pd.read_csv('labeled.csv')
dataFrame.head()


,comment,toxic
0,"Верблюдов-то за что? Дебилы, бл...\n",1.0
1,"Хохлы, это отдушина затюканого россиянина, мол...",1.0
2,Собаке - собачья смерть\n,1.0
3,"Страницу обнови, дебил. Это тоже не оскорблени...",1.0
4,"тебя не убедил 6-страничный пдф в том, что Скр...",1.0


In [36]:
train, test = train_test_split(data, test_size = 0.1, shuffle = True)
train.reset_index(inplace=True)
test.reset_index(inplace=True)

In [37]:
vectorizer1 = CountVectorizer(stop_words=['он','это','она'], min_df=1, max_df= 0.4, ngram_range=[1,2],lowercase=True)
X1 = vectorizer1.fit_transform(train.comment)
X_test1 = vectorizer1.transform(test.comment)
y1 = train.toxic.values
y_test1 = test.toxic.values

In [38]:
clf = LogisticRegression(C=0.1, class_weight='balanced')
clf.fit(X1, y1)
preds1 = clf.predict(X_test1)
print(classification_report(y_test1, preds1, zero_division=0))

              precision    recall  f1-score   support

         0.0       0.87      0.83      0.85       951
         1.0       0.70      0.77      0.73       491

    accuracy                           0.81      1442
   macro avg       0.78      0.80      0.79      1442
weighted avg       0.81      0.81      0.81      1442



In [39]:

probas = clf.predict_proba(X1)
preds = probas[:,1]
trainForLoop =train
for i in range (10):
    result = numpy.where(preds == numpy.amax(preds))
    index = result[0][0]
    print(i+1)
    print(trainForLoop.toxic[index])
    print(preds[index])
    print(trainForLoop.comment[index])
    preds = np.delete(preds, index)
    trainForLoop = trainForLoop.drop(index)
    

1
1.0
0.9999999997993707
Стас, никому, кроме тебя и армии твоих подсосов(которые представляют собой типичный дегенеративный биомусор, ведущийся на любые скандалы-интриги), твои ролики нахуй не нужны. Серьёзно, ты сделал новости с целью показать, что такое говно может делать любой, а аудитория осталась на том же уровне, ведь людям извне ты не интересен. Да ещё и просит не подписываться, чтобы такую-то годноту ложкой хлебать подольше. Ты обосрался, стал посмешищем для абсолютно всех ютуберов, которые не являются полными ебланами. Тот же Хованский не ссыт тебе на ебало только потому, что ты вертишься с ним в одной компании, иногда даже лично пересекаетесь. Приятно было слышать, как он говорил, что отстреливался бы от таких, как ты, из огнестрела, стараясь забрать с собой побольше коммунистов, когда они придут его оаскулачиапть? Он открыто хуесосил людей и за меньшие грехи. Сложи 2 и 2, как он к тебе относится на самом деле. После чего ты сделал ещё более смешной ролик, где истеришь как по

из десяти самые токсичные комменты, пять были отмечаны в оригиналном корпусе как токсичными, а пять были отмечаны как не токсичными. В основном, комменты с матерными словами были отмечены как токсичными (на пример, "нахуй" в первом, "блять" в девятом комменте) но есть и исключение. Например, второй, седьмой восьмой и десятый комменты не содерживает матерные слова, и на самом деле не токсичными. 

In [40]:
vectorizer2 = TfidfVectorizer(min_df=4, max_df=0.05, stop_words=['он', 'это', 'она', 'не', 'что'], ngram_range=[1,2], use_idf=True)
X2 = vectorizer2.fit_transform(train.comment)
X_test2 = vectorizer2.transform(test.comment)
y2 = train.toxic.values
y_test2 = test.toxic.values

In [41]:
clf2 = MultinomialNB(alpha =0.1, fit_prior =False)
clf2.fit(X2, y2)
preds2 = clf2.predict(X_test2)
print(classification_report(y_test2, preds2, zero_division=0))

              precision    recall  f1-score   support

         0.0       0.88      0.90      0.89       951
         1.0       0.80      0.76      0.78       491

    accuracy                           0.85      1442
   macro avg       0.84      0.83      0.83      1442
weighted avg       0.85      0.85      0.85      1442



In [42]:
probas2 = clf2.predict_proba(X2)
preds2 = probas2[:,1]
trainForLoop2 =train
for i in range (10):
    result = numpy.where(preds2 == numpy.amax(preds2))
    index = result[0][0]
    print(i+1)
    print(trainForLoop2.toxic[index])
    print(preds2[index])
    print(trainForLoop2.comment[index])
    preds2 = np.delete(preds2, index)
    trainForLoop2 = trainForLoop2.drop(index)

1
1.0
0.9999822864105564
Блеаадь, как же обидно когда создаешь тред, пародируешь речь ватников, случайно употребив слово из скрипта, и он скрывается у всех. Пересоздаю. Я много лет тут сижу с вами и обсераю пидорашек хоть я и сам один из них , смеюсь над их смертями, делаю фотожабы с обезьянами. Я-то думал это делают русские русофобы и украинцы, которых процентов 10, поэтому не относился как к чему-то иному как самокритике, самоиронии или справедливому негодованию из-за войны на Донбассе (в конце концов, я считаю украинцев своими братьями и их бугурт небезосновательным). Но меня нахуй осенило когда сначала вернули значки, а потом произошел расстрел мечети в НЗ. Тут до кучи блядских муслимов и они делают это вместе с нами (я думал они слишком беспросветно тупы для того, чтобы интересоваться политикой). Называют русских пидорашками, славщитом. Бляядь, вы действительно думаете вы лучше пидорашек? И вообще славян? Хорошо, я согласен, что кавказские мужчины часто выглядят лучше русских. Но 

Топ десятые комменты почти не совпадают с токсичными комментами прошлого классифакатора. Только коммент начиная с "Какие блять передергивания?" совпадает. Здесь тоже много матерные слова, но как в прошлом списке, есть исключение, где не понятно почему эти комменты может быть токсичными. Например, на самом деле, 4-й, 6-й и 9-й комменты не токсичными. 

## *Задание 4 (2 балла)

Для классификаторов LogisticRegression, Decision Trees, Naive Bayes, Random Forest найдите способ извлечь важность признаков для предсказания токсичного класса. Сопоставьте полученные числа со словами (или нграммами) в словаре и найдите топ - 5 "токсичных" слов для каждого из классификаторов. 

Важное требование: в топе не должно быть стоп-слов. Для этого вам нужно будет правильным образом настроить векторизацию.

In [ ]:
# ваш код